In [2]:
import sys
sys.path.append('../')
from util import corpus, tokenize, util, alignment, word2vec, retrieval
from tqdm.notebook import tqdm as tq
import pandas as pd

[nltk_data] Downloading package stopwords to /home/sudhi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sudhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = util.decompress_pickle('../../data/word2vec/epo_en/corpus/epo_18_20_df_wo_pre')
df

,name,titles,en,de
0,EP04714101NWB1.xml,de\tSYSTEME ZUR ÜBERWACHUNG UND AUTOMATISCHEN ...,"An automotive equipment control system 400, 10...","Ein Automobilgerätesteuerungssystem 400, 1000,..."
1,EP04714101NWB1.xml,de\tSYSTEME ZUR ÜBERWACHUNG UND AUTOMATISCHEN ...,"An automotive equipment control system 400, 10...","Ein Automobilgerätesteuerungssystem 400, 1000 ..."
2,EP04714101NWB1.xml,de\tSYSTEME ZUR ÜBERWACHUNG UND AUTOMATISCHEN ...,"An automotive equipment control system 400, 10...","Ein Automobilgerätesteuerungssystem 400, 1000 ..."
3,EP04714101NWB1.xml,de\tSYSTEME ZUR ÜBERWACHUNG UND AUTOMATISCHEN ...,"An automotive equipment control system 400, 10...","Ein Automobilgerätesteuerungssystem 400, 1000 ..."
4,EP04714101NWB1.xml,de\tSYSTEME ZUR ÜBERWACHUNG UND AUTOMATISCHEN ...,"An automotive equipment control system 400, 10...","Ein Automobilgerätesteuerungssystem 400, 1000 ..."
...,...,...,...,...
217860,EP19207198NWB1.xml,de\tELEKTROMAGNETISCHE VENTILVORRICHTUNG UND V...,The device according to any one of claims 1 to...,"Vorrichtung nach einem der Ansprüche 1 bis 7, ..."
217861,EP19207198NWB1.xml,de\tELEKTROMAGNETISCHE VENTILVORRICHTUNG UND V...,The device according to any one of claims 1 to...,"Vorrichtung nach einem der Ansprüche 1 bis 8, ..."
217862,EP19207198NWB1.xml,de\tELEKTROMAGNETISCHE VENTILVORRICHTUNG UND V...,The electromagnetic valve device according to ...,Elektromagnetische Ventilvorrichtung nach eine...
217863,EP19207198NWB1.xml,de\tELEKTROMAGNETISCHE VENTILVORRICHTUNG UND V...,A use of the electromagnetic valve device acco...,Verwendung der elektromagnetischen Ventilvorri...


In [6]:
patent_names = list(set(df.name.to_list()))
len(patent_names)

52354

In [3]:
df_grp = df.groupby("name")

In [5]:
pat_name_title = {}
for name in tq(patent_names):
    title = list(set(df_grp.get_group(name).titles.to_list()))
    if len(title) > 1:
        print(title, name)
    pat_name_title[name] = title

In [6]:
titles = []
for name in tq(patent_names):
    title = pat_name_title[name][0]
    splitted = title.split('\t')
    # print(splitted)
    
    de_title = splitted[splitted.index('de')+1]
    en_title = splitted[splitted.index('en')+1]
    # fr_title = splitted[splitted.index('fr')+1]
    
    titles.append(
                    {
                    'name': name,
                    'en_title': en_title.title(), 
                    'de_title': de_title.title()
                    }
                )

In [7]:
titles_df = pd.DataFrame(titles)
titles_df

,name,en_title,de_title
0,EP09003068NWB1.xml,Profile For Attaching Covering Material To A T...,Leiste Zum Anbringen Von Verdeckstoff An Einen...
1,EP15465506NWB1.xml,Layered Glass Assembly For Optical Light Beam ...,Verbundglasscheibenanordnung Für Optische Lich...
2,EP16191583NWB1.xml,Regenerative Brake Control Device,Steuerungsvorrichtung Für Eine Regenerative Br...
3,EP15757215NWB1.xml,Method For Monitoring And Diagnosing Component...,Verfahren Zur Überwachung Und Diagnose Von Kom...
4,EP17196476NWB1.xml,Closing System For A Door Of The Fuel Tank,Schliesssystem Für Eine Tür Des Kraftstofftanks
...,...,...,...
21304,EP12150302NWB1.xml,Electric Storage Device And Electric Storage A...,Elektrische Speichervorrichtung Und Elektrisch...
21305,EP10843128NWB1.xml,Battery Case For Vehicle,Batteriegehäuse Für Ein Fahrzeug
21306,EP17726594NWB1.xml,Vehicle Suspension With A Regenerative Hydraul...,Fahrzeugaufhängung Mit Regenerativem Hydraulis...
21307,EP13425023NWB1.xml,Window Construction,Fensterkonstruktion


In [8]:
patent_title = list(set(df_grp.get_group(patent_names[0]).titles.to_list()))
patent_title

["de\tLeiste zum Anbringen von Verdeckstoff an einen Verdeckspannbügel und Verdeck mit einer Leiste\ten\tProfile for attaching covering material to a tensioning bow and covering with a profile\tfr\tBaguette d'application de tissu textile sur un arceau de tension de capote et capote dotée d'une baguette\t"]

In [9]:
ind = 100

en_claims = df_grp.get_group(patent_names[ind]).en.to_list()
print('\n\n'.join(en_claims))

de_claims = df_grp.get_group(patent_names[ind]).de.to_list()
print('\n\n'.join(de_claims))

A constant mesh gearbox having a plurality of individually selectable forward speed ratios, and comprising a casing 20 defining an array of parallel rotational axes 3 35, said axes including an input axis 31, an output axis 35 and a plurality of intermediate axes 3 34, and gear pairs for driving between members rotatable on the input axis, the output axis and said intermediate axes, wherein a first shaft 12 is rotatable on one of said intermediate axes, a second shaft 14 is rotatable on another of said intermediate axes, a final drive gear 69 of an output shaft 15 on said output axis 35 is engaged with an output gear wheel 67 coupled for rotation with said first shaft 12 and an output gear wheel 68 coupled for rotation with said second shaft 14, thereby said first 12 and second 14 shaft are directly connected, a sleeve 74 is rotatable on said second shaft 14, and said sleeve and said first shaft are directly connected by a second gear wheel pair 75, 76, and

A constant mesh gearbox acc

## Check similarity of titles

In [10]:
import random

In [11]:
titles_df

,name,en_title,de_title
0,EP09003068NWB1.xml,Profile For Attaching Covering Material To A T...,Leiste Zum Anbringen Von Verdeckstoff An Einen...
1,EP15465506NWB1.xml,Layered Glass Assembly For Optical Light Beam ...,Verbundglasscheibenanordnung Für Optische Lich...
2,EP16191583NWB1.xml,Regenerative Brake Control Device,Steuerungsvorrichtung Für Eine Regenerative Br...
3,EP15757215NWB1.xml,Method For Monitoring And Diagnosing Component...,Verfahren Zur Überwachung Und Diagnose Von Kom...
4,EP17196476NWB1.xml,Closing System For A Door Of The Fuel Tank,Schliesssystem Für Eine Tür Des Kraftstofftanks
...,...,...,...
21304,EP12150302NWB1.xml,Electric Storage Device And Electric Storage A...,Elektrische Speichervorrichtung Und Elektrisch...
21305,EP10843128NWB1.xml,Battery Case For Vehicle,Batteriegehäuse Für Ein Fahrzeug
21306,EP17726594NWB1.xml,Vehicle Suspension With A Regenerative Hydraul...,Fahrzeugaufhängung Mit Regenerativem Hydraulis...
21307,EP13425023NWB1.xml,Window Construction,Fensterkonstruktion


In [12]:
en_titles = titles_df.en_title.to_list()
names = titles_df.name.to_list()
c = list(zip(names, en_titles))

In [13]:
random.shuffle(c)

In [14]:
names_shuff, en_titles_shuff = zip(*c)

In [15]:
test_names = names_shuff[:1000]
test_en_titles = en_titles_shuff[:1000]

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

01:23:01: Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
01:23:06: Use pytorch device: cuda


In [17]:
vec = model.encode(test_en_titles, convert_to_tensor=True)

In [18]:
import torch
import torch.nn.functional as F

q_ind = 0
sim = F.cosine_similarity(vec[0].unsqueeze(0), vec, dim=1)

In [19]:
test_data = []
for ind in range(len(test_en_titles)):
    sim = F.cosine_similarity(vec[ind].unsqueeze(0), vec, dim=1)
    
    sim_values = torch.sort(sim, descending=True).values
    pat_indices = torch.sort(sim, descending=True).indices
    
    # get the irrelevant indices. the last 5 ones.
    irr_indices = pat_indices.cpu().numpy()[-5:]
    
    # get the relevant indices. threshold with 0.85
    rel_indices = pat_indices[sim_values > 0.85].cpu().numpy()
    
    test_data.append(
        {
            'name':test_names[ind],
            'title':test_en_titles[ind],
            
            'rel_idx':rel_indices,
            'irr_idx':irr_indices,
            
            'rel_titles':[test_en_titles[idx] for idx in rel_indices],
            'irr_titles':[test_en_titles[idx] for idx in irr_indices],
            
            'rel_names':[test_names[idx] for idx in rel_indices],
            'irr_names':[test_names[idx] for idx in irr_indices],            
        }
    )

In [20]:
sim_values = torch.sort(sim, descending=True).values
pat_indices = torch.sort(sim, descending=True).indices

irr_indices = pat_indices.cpu().numpy()[-5:]
irr_indices

array([621, 473, 522,   4, 528])

In [21]:
sim_values[sim_values > 0.85]

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.8691],
       device='cuda:0')

In [22]:
rel_indices = pat_indices[sim_values > 0.85].cpu().numpy()

In [23]:
print(test_en_titles[q_ind])

Wiper Blade Adapter, In Particular For A Motor Vehicle Wiper Device


In [24]:
print('Relevant', [test_en_titles[idx] for idx in rel_indices])

Relevant ['Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Vehicle', 'Straddled Vehicle']


In [25]:
[test_en_titles[idx] for idx in irr_indices]

['Method For Damping Electromechanical Oscillations In An Electromechanical System And Oscillation Damping System For Employing Such Method',
 'Process For The Preparation Of Precipitated Silicas, Precipitated Silicas And Their Uses, In Particular For The Reinforcement Of Polymers',
 'Systems And Methods For The Detection Of Implicit Gestures',
 'Hollow Profiled Element Having Depressions',
 'Apparatus For Obtaining Predetermined Damping Characteristics And A Simultaneous Energy Recovery In Regenerative Shock Absorbers And Corresponding Method']

In [31]:
test_data[0]

{'name': 'EP11787853NWB1.xml',
 'title': 'Wiper Blade Adapter, In Particular For A Motor Vehicle Wiper Device',
 'rel_idx': array([  0, 114]),
 'irr_idx': array([622, 568, 522,  27,   4]),
 'rel_titles': ['Wiper Blade Adapter, In Particular For A Motor Vehicle Wiper Device',
  'Wiper Blade Device'],
 'irr_titles': ['Driving Prediction With A Deep Neural Network',
  'Tolerance Gap Flag',
  'Systems And Methods For The Detection Of Implicit Gestures',
  'Communication System And Communication Method',
  'Hollow Profiled Element Having Depressions'],
 'rel_names': ['EP11787853NWB1.xml', 'EP12809219NWB1.xml'],
 'irr_names': ['EP17160823NWB1.xml',
  'EP15196468NWB1.xml',
  'EP15716684NWB1.xml',
  'EP13734831NWB1.xml',
  'EP16700859NWB1.xml']}

In [32]:
def get_claims_from_name(name):
    en_claims = df_grp.get_group(name).en.to_list()
    de_claims = df_grp.get_group(name).de.to_list()
    
    return en_claims, de_claims

In [46]:
test = []
for patent in test_data:
    name = patent['name']
    rel_names = patent['rel_names']
    irr_names = patent['irr_names']
    
    title = patent['title']
    rel_titles = patent['rel_titles']
    irr_titles = patent['irr_titles']
    
    own_claims_en, own_claims_de = get_claims_from_name(name)
    
    rel_claims_en, rel_claims_de = [],  []
    for r in rel_names:
        en, de = get_claims_from_name(r)
        rel_claims_en.extend(en)
        rel_claims_de.extend(de)
        
    irr_claims_en, irr_claims_de = [],  []
    for r in irr_names:
        en, de = get_claims_from_name(r)
        irr_claims_en.extend(en)
        irr_claims_de.extend(de)        
    
    test.append(
        {
            'name':name,
            'query':title,
            
            'rel_titles':rel_titles,
            'irr_titles':irr_titles,
            
            'own_claims_en':own_claims_en,
            'own_claims_de':own_claims_de,
            
            'rel_claims_en':rel_claims_en,
            'rel_claims_de':rel_claims_de,
            
            'irr_claims_en':irr_claims_en,
            'irr_claims_de':irr_claims_de,
        }
    )

test_df = pd.DataFrame(test)
test_df

,name,query,rel_titles,irr_titles,own_claims_en,own_claims_de,rel_claims_en,rel_claims_de,irr_claims_en,irr_claims_de
0,EP11787853NWB1.xml,"Wiper Blade Adapter, In Particular For A Motor...","[Wiper Blade Adapter, In Particular For A Moto...",[Driving Prediction With A Deep Neural Network...,"[Wiper blade adapter, in particular for a moto...","[Wischblattadapter, insbesondere für eine Kraf...","[Wiper blade adapter, in particular for a moto...","[Wischblattadapter, insbesondere für eine Kraf...",[Method for assisting a driver to drive a vehi...,[Verfahren zum Unterstützen eines Fahrers beim...
1,EP14001859NWB1.xml,Method For Producing A Suspension Arm And Susp...,[Method For Producing A Suspension Arm And Sus...,"[Caster Indicator, Tolerance Gap Flag, Psi5 In...",[Method for producing a suspension link 1 for ...,[Verfahren zur Herstellung eines Fahrwerkslenk...,[Method for producing a suspension link 1 for ...,[Verfahren zur Herstellung eines Fahrwerkslenk...,[A caster indicator 200 comprising plural leve...,"[Rollenanzeige 200, welche Folgendes aufweist ..."
2,EP14189324NWB1.xml,Removable Luggage Box For Storage In A Luggage...,[Removable Luggage Box For Storage In A Luggag...,[Illumination For Detecting Raindrops On A Pan...,[A motor vehicle 1 having a confined space 2 o...,"[Kraftfahrzeug 1 mit einem begrenzten Raum 2, ...",[A motor vehicle 1 having a confined space 2 o...,"[Kraftfahrzeug 1 mit einem begrenzten Raum 2, ...",[A device for detecting rain on a pane 2 compr...,[Vorrichtung zur Erkennung von Regen auf einer...
3,EP16725464NWB1.xml,Electrical Steering Column Lock,"[Electrical Steering Column Lock, A Steering C...","[Nonwoven Textile Made From Short Fibers, Biax...",[Electrical steering column lock for locking a...,[Elektrisches Lenkradschloss zum Verriegeln un...,[Electrical steering column lock for locking a...,[Elektrisches Lenkradschloss zum Verriegeln un...,"[A method of forming a nonwoven material, the ...",[Verfahren zur Herstellung eines Vliesmaterial...
4,EP16700859NWB1.xml,Hollow Profiled Element Having Depressions,[Hollow Profiled Element Having Depressions],[Method For Managing The Supply Of Electrical ...,"[A hollow profiled element 1, comprising a plu...",[Hohlprofil 1 mit mehreren in einer Längsricht...,"[A hollow profiled element 1, comprising a plu...",[Hohlprofil 1 mit mehreren in einer Längsricht...,[A method for managing the supply of electrica...,[Verfahren zur Verwaltung der Stromversorgung ...
...,...,...,...,...,...,...,...,...,...,...
995,EP14196574NWB1.xml,Assembly Fixture For Supporting A Fuselage Ass...,[Assembly Fixture For Supporting A Fuselage As...,"[Receptacle, Illumination For Detecting Raindr...",[A method for building a fuselage assembly 114...,[Verfahren zum Bau einer Rumpfbaugruppe 114 11...,[A method for building a fuselage assembly 114...,[Verfahren zum Bau einer Rumpfbaugruppe 114 11...,[Receiving device for arranging in a motor veh...,[Aufnahmevorrichtung zum Anordnen in einem Kra...
996,EP12156295NWB1.xml,Turn Signal Switch Device,"[Turn Signal Switch Device, Turn Signal Indica...","[Biaxially Stretched Laminated Polyester Film,...",[A turn signal switch device comprising an ope...,"[Fahrtrichtungsanzeigen-Schaltervorrichtung, d...",[A turn signal switch device comprising an ope...,"[Fahrtrichtungsanzeigen-Schaltervorrichtung, d...",[A biaxially stretched laminated polyester fil...,"[Biaxial gestreckte, laminierte Polyesterfolie..."
997,EP18166119NWB1.xml,Convertible Vehicle With A Retaining Element F...,[Convertible Vehicle With A Retaining Element ...,"[Child Protector Mechanism, Fixation Of The Ba...",[A convertible vehicle 1 comprising a hood 2 w...,[Cabriolet-Fahrzeug 1 mit einem in einem hinte...,[A convertible vehicle 1 comprising a hood 2 w...,[Cabriolet-Fahrzeug 1 mit einem in einem hinte...,"[A child protector mechanism 10, comprising a ...","[Kindersicherungsmechanismus 10, der Folgendes..."
998,EP16779103NWB1.xml,Brake Tube Connector And Brake Tube Connector .

In [48]:
util.compress_pickle('test_df_1000_queries', test_df)

In [4]:
test_df = util.decompress_pickle('../../data/test/test_df_1000_queries')
test_df

,name,query,rel_titles,irr_titles,own_claims_en,own_claims_de,rel_claims_en,rel_claims_de,irr_claims_en,irr_claims_de
0,EP11787853NWB1.xml,"Wiper Blade Adapter, In Particular For A Motor...","[Wiper Blade Adapter, In Particular For A Moto...",[Driving Prediction With A Deep Neural Network...,"[Wiper blade adapter, in particular for a moto...","[Wischblattadapter, insbesondere für eine Kraf...","[Wiper blade adapter, in particular for a moto...","[Wischblattadapter, insbesondere für eine Kraf...",[Method for assisting a driver to drive a vehi...,[Verfahren zum Unterstützen eines Fahrers beim...
1,EP14001859NWB1.xml,Method For Producing A Suspension Arm And Susp...,[Method For Producing A Suspension Arm And Sus...,"[Caster Indicator, Tolerance Gap Flag, Psi5 In...",[Method for producing a suspension link 1 for ...,[Verfahren zur Herstellung eines Fahrwerkslenk...,[Method for producing a suspension link 1 for ...,[Verfahren zur Herstellung eines Fahrwerkslenk...,[A caster indicator 200 comprising plural leve...,"[Rollenanzeige 200, welche Folgendes aufweist ..."
2,EP14189324NWB1.xml,Removable Luggage Box For Storage In A Luggage...,[Removable Luggage Box For Storage In A Luggag...,[Illumination For Detecting Raindrops On A Pan...,[A motor vehicle 1 having a confined space 2 o...,"[Kraftfahrzeug 1 mit einem begrenzten Raum 2, ...",[A motor vehicle 1 having a confined space 2 o...,"[Kraftfahrzeug 1 mit einem begrenzten Raum 2, ...",[A device for detecting rain on a pane 2 compr...,[Vorrichtung zur Erkennung von Regen auf einer...
3,EP16725464NWB1.xml,Electrical Steering Column Lock,"[Electrical Steering Column Lock, A Steering C...","[Nonwoven Textile Made From Short Fibers, Biax...",[Electrical steering column lock for locking a...,[Elektrisches Lenkradschloss zum Verriegeln un...,[Electrical steering column lock for locking a...,[Elektrisches Lenkradschloss zum Verriegeln un...,"[A method of forming a nonwoven material, the ...",[Verfahren zur Herstellung eines Vliesmaterial...
4,EP16700859NWB1.xml,Hollow Profiled Element Having Depressions,[Hollow Profiled Element Having Depressions],[Method For Managing The Supply Of Electrical ...,"[A hollow profiled element 1, comprising a plu...",[Hohlprofil 1 mit mehreren in einer Längsricht...,"[A hollow profiled element 1, comprising a plu...",[Hohlprofil 1 mit mehreren in einer Längsricht...,[A method for managing the supply of electrica...,[Verfahren zur Verwaltung der Stromversorgung ...
...,...,...,...,...,...,...,...,...,...,...
995,EP14196574NWB1.xml,Assembly Fixture For Supporting A Fuselage Ass...,[Assembly Fixture For Supporting A Fuselage As...,"[Receptacle, Illumination For Detecting Raindr...",[A method for building a fuselage assembly 114...,[Verfahren zum Bau einer Rumpfbaugruppe 114 11...,[A method for building a fuselage assembly 114...,[Verfahren zum Bau einer Rumpfbaugruppe 114 11...,[Receiving device for arranging in a motor veh...,[Aufnahmevorrichtung zum Anordnen in einem Kra...
996,EP12156295NWB1.xml,Turn Signal Switch Device,"[Turn Signal Switch Device, Turn Signal Indica...","[Biaxially Stretched Laminated Polyester Film,...",[A turn signal switch device comprising an ope...,"[Fahrtrichtungsanzeigen-Schaltervorrichtung, d...",[A turn signal switch device comprising an ope...,"[Fahrtrichtungsanzeigen-Schaltervorrichtung, d...",[A biaxially stretched laminated polyester fil...,"[Biaxial gestreckte, laminierte Polyesterfolie..."
997,EP18166119NWB1.xml,Convertible Vehicle With A Retaining Element F...,[Convertible Vehicle With A Retaining Element ...,"[Child Protector Mechanism, Fixation Of The Ba...",[A convertible vehicle 1 comprising a hood 2 w...,[Cabriolet-Fahrzeug 1 mit einem in einem hinte...,[A convertible vehicle 1 comprising a hood 2 w...,[Cabriolet-Fahrzeug 1 mit einem in einem hinte...,"[A child protector mechanism 10, comprising a ...","[Kindersicherungsmechanismus 10, der Folgendes..."
998,EP16779103NWB1.xml,Brake Tube Connector And Brake Tube Connector .

In [9]:
test_df.keys()

Index(['name', 'query', 'rel_titles', 'irr_titles', 'own_claims_en',
       'own_claims_de', 'rel_claims_en', 'rel_claims_de', 'irr_claims_en',
       'irr_claims_de'],
      dtype='object')

In [10]:
test_df.columns = ['name', 'query_word', 'rel_titles', 'irr_titles', 'own_claims_en',
       'own_claims_de', 'rel_claims_en', 'rel_claims_de', 'irr_claims_en',
       'irr_claims_de']

In [14]:
for query, rel_titles, irr_titles in zip(
                                        test_df.query_word,
                                        test_df.rel_titles,
                                        test_df.irr_titles
                                    ):
    
    print('Query:', query)
    print('Relevant:', rel_titles)
    print('Irrelevant:', irr_titles)
    print()

Query: Wiper Blade Adapter, In Particular For A Motor Vehicle Wiper Device
Relevant: ['Wiper Blade Adapter, In Particular For A Motor Vehicle Wiper Device', 'Wiper Blade Device']
Irrelevant: ['Driving Prediction With A Deep Neural Network', 'Tolerance Gap Flag', 'Systems And Methods For The Detection Of Implicit Gestures', 'Communication System And Communication Method', 'Hollow Profiled Element Having Depressions']

Query: Method For Producing A Suspension Arm And Suspension Arm For A Vehicle
Relevant: ['Method For Producing A Suspension Arm And Suspension Arm For A Vehicle']
Irrelevant: ['Caster Indicator', 'Tolerance Gap Flag', 'Psi5 Interface Universally Usable For Asynchronous And Synchronous Data Transfer', 'Systems And Methods For The Detection Of Implicit Gestures', 'Hollow Profiled Element Having Depressions']

Query: Removable Luggage Box For Storage In A Luggage Or Trunk Space Of A Motor Vehicle, And Especially Of A Hatchback Or Station Wagon Automobile
Relevant: ['Removable

In [6]:
test_sm_df = test_df[:50]

In [7]:
all_claims_en = []
for claims in test_sm_df.own_claims_en:
    for claim in claims:
        all_claims_en.append(claim)       
        
for claims in test_sm_df.rel_claims_en:
    for claim in claims:
        all_claims_en.append(claim)
        
for claims in test_sm_df.irr_claims_en:
    for claim in claims:
        all_claims_en.append(claim) 
        
        
all_claims_de = []
        
for claims in test_sm_df.own_claims_de:
    for claim in claims:
        all_claims_de.append(claim) 
        
for claims in test_sm_df.rel_claims_de:
    for claim in claims:
        all_claims_de.append(claim)          
        
for claims in test_sm_df.irr_claims_de:
    for claim in claims:
        all_claims_de.append(claim)   
        
len(all_claims_en), len(all_claims_de)

(4639, 4639)

In [8]:
util.compress_pickle('test_en_claims_first_50_queries', all_claims_en)
util.compress_pickle('test_de_claims_first_50_queries', all_claims_de)